## Evaluate.ipynb
#### **This notebook evaluates the accuracy of just one model on the IEMOCAP dataset ground truths.**

- **Transcription Model (S2T):** gemini-flash-1.5
- Source: Google

## Setup

In [1]:
# Run
SEED = 22
FOLD = 0

# Models
TRANSCRIPTION_MODEL_NAME = 'models/gemini-1.5-flash'

# Flag to enable parsing of arguments when converted to script. Set true after converting
PYTHON_SCRIPT = False

### For Conversion to .py file

In [ ]:
if PYTHON_SCRIPT:

    import argparse

    parser = argparse.ArgumentParser(description="")
    parser.add_argument('--seed',default=2021,type=int)
    parser.add_argument('--fold',type=int,required=True)
    parser.add_argument('--remap',type=bool,required=True)
    parser.add_argument('--threshold',type=float,required=False)
    parser.add_argument('--mode',required=False)
    parser.add_argument('--flip',type=bool,required=False)

    # Parse the arguments
    args = parser.parse_args()

    # Run
    SEED = args.seed
    FOLD = args.fold
    RUN_REMAP = args.remap
    THRESHOLD = args.threshold
    MODE = args.mode
    FLIP = args.flip

### Folders

In [2]:
# Filepaths
OUTPUT_FOLDER = f'./fold_{FOLD}'
LOG_OUTPUT = OUTPUT_FOLDER + f'/fold-{FOLD}.log'

### Dataset Setup

In [3]:
# Dataset
TSV = r'C:\Users\ChenYi\Downloads\AAI3001_Project\labels\IEMOCAP_4.tsv'
AUDIO_DIRECTORY = r'C:\Users\ChenYi\Downloads\AAI3001_Project\small-project\IEMOCAP_full_release_audio'

### Select GPUs (For multi-GPU setup)

In [4]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Logger

In [5]:
import logging
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter("[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s")
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(fh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    sh = logging.StreamHandler()                                                                                                                                                                                   
    sh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(sh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    return logger
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
logger = get_logger(LOG_OUTPUT)

## Imports

In [6]:
import google.generativeai as genai
import pathlib
import time
import csv
import torch
import random
import librosa
import torchaudio
import contractions
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset, DatasetDict
from torchaudio import functional as audioF
from torchaudio.transforms import Resample
from torchaudio.compliance import kaldi
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report, recall_score, accuracy_score

[2025-01-12 09:07:26,055][config.py][line:54][INFO] PyTorch version 2.4.1+cu124 available.
[2025-01-12 09:07:26,058][config.py][line:101][INFO] TensorFlow version 2.18.0 available.


### Log Details

In [7]:
logger.info("----- Parameters -----")
logger.info(f"Seed: {SEED}")
logger.info(f"Fold: {FOLD}")

[2025-01-12 09:07:26,276][475919163.py][line:1][INFO] ----- Parameters -----
[2025-01-12 09:07:26,277][475919163.py][line:2][INFO] Seed: 22
[2025-01-12 09:07:26,278][475919163.py][line:3][INFO] Fold: 0


### Provided Code

In [8]:
class Pad_trunc_wav(nn.Module):
    def __init__(self, max_len: int = 6*16000):
        super(Pad_trunc_wav, self).__init__()
        self.max_len = max_len
    def forward(self,x):
        shape = x.shape
        length = shape[1]
        if length < self.max_len:
            multiple = self.max_len//length+1
            x_tmp = torch.cat((x,)*multiple, axis=1)
            x_new = x_tmp[:,0:self.max_len]
        else:
            x_new = x[:,0:self.max_len]
        return x_new

In [9]:
def setup_seed(seed=2021):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
setup_seed(SEED)

### Setup Gemini

In [10]:
genai.configure(api_key="")

# Initialize a Gemini model appropriate for your use case.
model = genai.GenerativeModel('models/gemini-1.5-flash')

In [11]:
prompt = "Please provide an English transcript of the audio file. If there is an error with the file, or you are not able to transcribe it, respond with a single space character like ' '"
CURRENT_TOKEN_COUNT = 0 # Manually updated entry to keep track of API usage

In [12]:
logger.info("----- Gemini API -----")
logger.info(f"Model: {TRANSCRIPTION_MODEL_NAME}")
logger.info(f"Current Token Count: {CURRENT_TOKEN_COUNT}")
logger.info("--------------------")

[2025-01-12 09:07:26,313][1209516506.py][line:1][INFO] ----- Gemini API -----
[2025-01-12 09:07:26,314][1209516506.py][line:2][INFO] Model: models/gemini-1.5-flash
[2025-01-12 09:07:26,314][1209516506.py][line:3][INFO] Current Token Count: 167193
[2025-01-12 09:07:26,315][1209516506.py][line:4][INFO] --------------------


### Dataset & Loading

In [13]:
class Mydataset(Dataset):
    def __init__(self, mode='train', max_len=6, seed=2021, fold=0, data_path=TSV, audio_dir=AUDIO_DIRECTORY):
        self.mode = mode
        data_all = pd.read_csv(data_path, sep='\t')
        SpkNames = np.unique(data_all['speaker'])  # ['Ses01F', 'Ses01M', ..., 'Ses05M']
        self.data_info = self.split_dataset(data_all, fold, SpkNames, mode)
        self.get_audio_dir_path = os.path.join(audio_dir)
        self.pad_trunc = Pad_trunc_wav(max_len * 16000)
         
        # Label encoding
        self.label = self.data_info['label'].astype('category').cat.codes.values
        self.ClassNames = np.unique(self.data_info['label'])
        self.NumClasses = len(self.ClassNames)
        self.weight = 1 / torch.tensor([(self.label == i).sum() for i in range(self.NumClasses)]).float()

    def get_classname(self):
        return self.ClassNames
    
    # Updated split_dataset function using fold
    
    def split_dataset(self, df_all, fold, speakers, mode):
        
        spk_len = len(speakers)
        test_idx = np.array(df_all['speaker']==speakers[fold%spk_len])
        if fold%2==0:
            val_idx = np.array(df_all['speaker']==speakers[(fold+1)%spk_len])
        else:
            val_idx = np.array(df_all['speaker']==speakers[(fold-1)%spk_len])
        train_idx = True^(test_idx+val_idx)
        train_data_info = df_all[train_idx].reset_index(drop=True)
        val_data_info = df_all[val_idx].reset_index(drop=True)
        test_data_info = df_all[test_idx].reset_index(drop=True)

        if self.mode == 'train':
            data_info = train_data_info
        elif self.mode == 'val':
            data_info = val_data_info
        elif self.mode == 'test':
            data_info = test_data_info
        else:
            data_info = df_all
        
        logger.info(f"Mode: {mode} Fold: {fold}")
        return data_info

    def pre_process(self, wav):
        
        if self.mode == 'test': 
            return wav
        else:
            wav = self.pad_trunc(wav)
            return wav

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        # Load the raw waveform from file using data_info to get filenames
        wav_path = os.path.join(self.get_audio_dir_path, self.data_info['filename'][idx]) + '.wav'
        wav, sample_rate = torchaudio.load(wav_path)

        # Preprocess the waveform (e.g., pad/truncate if needed)
        wav = self.pre_process(wav)

        # Apply Wav2Vec2 feature extractor
        inputs = feature_extractor(
            wav.squeeze().numpy(),  # Convert PyTorch tensor to numpy array
            sampling_rate=sample_rate,
            return_tensors="pt",  # Return PyTorch tensors
            padding=True  # Optionally pad to a fixed length
        )

        label = self.label[idx]

        # Return the processed input values and the label
        return {
            'input_values': inputs['input_values'].squeeze(0),  # Remove extra batch dimension
            'labels': torch.tensor(label, dtype=torch.long)}

In [14]:
# Instantiate datasets
test_dataset = Mydataset(mode='test', max_len=6, fold=FOLD)

logger.info("Dataset Loaded")

[2025-01-12 09:07:26,345][1700717295.py][line:43][INFO] Mode: test Fold: 0
[2025-01-12 09:07:26,347][1146238807.py][line:4][INFO] Dataset Loaded


In [15]:
# Put test information into a dataframe for later use
data_info = test_dataset.data_info
test_dataframe = data_info[['filename', 'label']].copy()
test_dataframe['filepath'] = test_dataframe['filename'].apply(
    lambda x: os.path.join(test_dataset.get_audio_dir_path, f"{x}.wav"))

## Perform Inference and Obtain Transcriptions

#### Function to clean up transcription to match what is required for compute_wer.py
- Remove full stop
- Remove all punctuations except apostrophe
- Change all to UPPERCASE

In [16]:
def format_transcription(ID, count, text):
    
    original_text = text  # Store the original text for comparison
    
    # Remove all non-ASCII characters
    clean_text = ''.join(c for c in text if ord(c) < 128)
    
    # 2. Remove full stops (periods)
    clean_text = clean_text.replace('.', '')
    
    # 3. Remove all punctuations except apostrophes
    #    (everything that is not alphanumeric or apostrophe => replaced by space)
    clean_text = ''.join(char if char.isalnum() or char == "'" else ' ' 
                         for char in clean_text)
    
    # 4. Convert to uppercase
    clean_text = clean_text.upper()

    # Alert if any formatting was done
    # (i.e., if the final clean_text differs from the *original*, unmodified text)
    if clean_text != original_text:
        logger.info(f"Transcription of file {count} '{ID}' has been formatted")
        logger.info(f"ORIGINAL: {original_text}")
        logger.info(f"FORMATTED: {clean_text}")
    
    return clean_text

In [17]:
def predict(test_dataframe):

    results = []
    total = test_dataframe.shape[0]
    count = 1
    total_tokens = CURRENT_TOKEN_COUNT

    # Iterate over each audio file in the test folder
    for index, row in test_dataframe.iterrows():

        # Display progress
        print(f'File {count} of {total}', end='\r')
        count += 1

        # Get audio filename
        filename = row['filename'] + '.wav'
        audio_file = os.path.join(AUDIO_DIRECTORY, filename)
        
        response = model.generate_content([
            prompt,
            {
                "mime_type": "audio/wav",
                "data": pathlib.Path(audio_file).read_bytes()
            }
        ])
        
        total_token_count = response.usage_metadata.total_token_count
        total_tokens += total_token_count
        
        text = response.candidates[0].content.parts[0].text
        
        #print(f"[Total Tokens Used] {total_tokens}")
        #print(f"[Text] {text}")
        
        # Perform cleaning
        formatted_transcription = format_transcription(filename, count, text)

        # Extract the filename without the extension
        filename = os.path.splitext(os.path.basename(audio_file))[0]

        # Append the result to the list
        results.append([filename,
                        formatted_transcription])
        
        # Log tokens usage
        if (count%55 == 0):
            logger.info(f"------ Gemini API Usage Metrics ------")
            logger.info(f"[Files Processed] {count} of {total}")
            logger.info(f"[Total Tokens] {total_tokens}")
            logger.info(f"--------------------------------------")
            

    logger.info(f"Done processing {total} files")
    
    logger.info(f"------ Gemini API Usage Metrics ------")
    logger.info(f"[Files Processed] {count-1} of {total}")
    logger.info(f"[Total Tokens] {total_tokens}")
    logger.info(f"--------------------------------------")

    # Write the results to a CSV file
    global CSV_FILEPATH
    CSV_FILEPATH = os.path.join(OUTPUT_FOLDER, "transcriptions.csv")

    with open(CSV_FILEPATH, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'transcription'])
        writer.writerows(results)

    logger.info(f"Transcriptions saved to {CSV_FILEPATH}")
    return CSV_FILEPATH

In [18]:
predict(test_dataframe)

[2025-01-12 09:07:28,259][850594287.py][line:22][INFO] Transcription of file Ses01F_script01_3_F000.wav has been formatted
[2025-01-12 09:07:28,260][850594287.py][line:23][INFO] ORIGINAL: You're the only one who still loves his parents.

[2025-01-12 09:07:28,261][850594287.py][line:24][INFO] FORMATTED: YOU'RE THE ONLY ONE WHO STILL LOVES HIS PARENTS 


[2025-01-12 09:07:29,299][850594287.py][line:22][INFO] Transcription of file Ses01F_script01_3_F001.wav has been formatted
[2025-01-12 09:07:29,300][850594287.py][line:23][INFO] ORIGINAL: It's not so bad.  It's a good thing.

[2025-01-12 09:07:29,300][850594287.py][line:24][INFO] FORMATTED: IT'S NOT SO BAD  IT'S A GOOD THING 


[2025-01-12 09:07:30,503][850594287.py][line:22][INFO] Transcription of file Ses01F_script01_3_F002.wav has been formatted
[2025-01-12 09:07:30,504][850594287.py][line:23][INFO] ORIGINAL: You know, it's lovely here. The air is sweet.

[2025-01-12 09:07:30,505][850594287.py][line:24][INFO] FORMATTED: YOU KNOW  IT'S LOVELY HERE THE AIR IS SWEET 


[2025-01-12 09:07:31,674][850594287.py][line:22][INFO] Transcription of file Ses01F_script01_3_F003.wav has been formatted
[2025-01-12 09:07:31,675][850594287.py][line:23][INFO] ORIGINAL: I'm sorry, no.  But I'm I can't stay.

[2025-01-12 09:07:31,675][850594287.py][line:24][INFO] FORMATTED: I'M SORRY  NO  BUT I'M I CAN'T STAY 


[2025-01-12 09:07:32,646][850594287.py][line:22][INFO] Transcription of file Ses01F_script01_3_F004.wav has been formatted
[2025-01-12 09:07:32,647][850594287.py][line:23][INFO] ORIGINAL: For one thing, your mother most assuredly told me to go.

[2025-01-12 09:07:32,648][850594287.py][line:24][INFO] FORMATTED: FOR ONE THING  YOUR MOTHER MOST ASSUREDLY TOLD ME TO GO 


...

[2025-01-12 09:17:26,434][850594287.py][line:22][INFO] Transcription of file Ses01F_script03_1_F022.wav has been formatted
[2025-01-12 09:17:26,435][850594287.py][line:23][INFO] ORIGINAL: Oh, thank you, darling.  The same applies to you too. Except that if you so much as look at another woman, I'll kill you.

[2025-01-12 09:17:26,435][850594287.py][line:24][INFO] FORMATTED: OH  THANK YOU  DARLING  THE SAME APPLIES TO YOU TOO EXCEPT THAT IF YOU SO MUCH AS LOOK AT ANOTHER WOMAN  I'LL KILL YOU 


[2025-01-12 09:17:27,397][850594287.py][line:22][INFO] Transcription of file Ses01F_script03_1_F024.wav has been formatted
[2025-01-12 09:17:27,398][850594287.py][line:23][INFO] ORIGINAL: Oh, Charles! Ooh, that was his name. He did wiggle so beautifully.

[2025-01-12 09:17:27,399][850594287.py][line:24][INFO] FORMATTED: OH  CHARLES  OOH  THAT WAS HIS NAME HE DID WIGGLE SO BEAUTIFULLY 


[2025-01-12 09:17:28,587][850594287.py][line:22][INFO] Transcription of file Ses01F_script03_1_F029.wav has been formatted
[2025-01-12 09:17:28,589][850594287.py][line:23][INFO] ORIGINAL: The manager came in. He found us rolling about the floor, biting and scratching like panthers!  Oh! I'll never forget it. How utterly, utterly ridiculous!  Ppppppp

[2025-01-12 09:17:28,591][850594287.py][line:24][INFO] FORMATTED: THE MANAGER CAME IN HE FOUND US ROLLING ABOUT THE FLOOR  BITING AND SCRATCHING LIKE PANTHERS   OH  I'LL NEVER FORGET IT HOW UTTERLY  UTTERLY RIDICULOUS   PPPPPPP 


[2025-01-12 09:17:29,522][850594287.py][line:22][INFO] Transcription of file Ses01F_script03_1_F030.wav has been formatted
[2025-01-12 09:17:29,524][850594287.py][line:23][INFO] ORIGINAL: We were very much younger then.  Mm, and very much sillier.

[2025-01-12 09:17:29,525][850594287.py][line:24][INFO] FORMATTED: WE WERE VERY MUCH YOUNGER THEN  MM  AND VERY MUCH SILLIER 
[2025-01-12 09:17:29,526][2649188033.py][line:54][INFO] Done processing 528 files
[2025-01-12 09:17:29,527][2649188033.py][line:56][INFO] ------ Gemini API Usage Metrics ------
[2025-01-12 09:17:29,528][2649188033.py][line:57][INFO] [Files Processed] 528 of 528
[2025-01-12 09:17:29,529][2649188033.py][line:58][INFO] [Total Tokens] 272338
[2025-01-12 09:17:29,530][2649188033.py][line:59][INFO] --------------------------------------
[2025-01-12 09:17:29,534][2649188033.py][line:70][INFO] Transcriptions saved to ./fold_0\transcriptions.csv


'./fold_0\\transcriptions.csv'

## Convert CSV into TXT file
- Optional step to convert into TXT file for use in compute_wer.py
- This function can also be found in notebooks/TranscriptionFormatter.ipynb, where you can optionaly convert each fold at once

In [19]:
# Input and output file paths
input_csv = CSV_FILEPATH  # Replace with your CSV file path
output_txt = OUTPUT_FOLDER + "/transcriptions.txt"  # Replace with your desired TXT file path

In [20]:
# Open the CSV file and read its content
def convert_csv_to_txt(input_csv, output_txt):
    with open(input_csv, mode="r", encoding="utf-8") as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader, None)  # Skip header if present
        with open(output_txt, mode="w", encoding="utf-8") as txt_file:
            for row in csv_reader:
                # Combine ID and Transcript with a space separator
                txt_file.write(f"{row[0]} {row[1]}\n")
    
convert_csv_to_txt(input_csv, output_txt)
print(f"Converted and formatted {input_csv} to {output_txt} for use in computing word error rate.")

Converted and formatted ./fold_0\transcriptions.csv to ./fold_0/transcriptions.txt for use in computing word error rate.
